필요한 것
dotenv
embedding

index_name
database

llm
prompt
retriever

ai message
qa_chain으로 보강

In [ ]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_openai import ChatOpenAI
from langchain import hub
# query -> 직장인 -> 거주자로 바꾸는 chain을 추가를 위한 라이브러리
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate


load_dotenv()

embeddings = OpenAIEmbeddings(model = 'text-embedding-3-large')
index_name = 'tax-markdown-index'

database = PineconeVectorStore( index=index_name, embedding=embeddings)
prompt = hub.pull("rlm/rag-prompt")
llm = ChatOpenAI(model = 'gpt-4o-mini', temperature = 0)
chain_type_kwargs = {"prompt":prompt}
retriever = database.as_retriever(search_kwargs={"k":4})
ai_message = llm.invoke(prompt)

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever = database.as_retriever(),
    chain_type_kwargs = {"prompt":prompt}
)

ai_message = qa_chain({"query":query})


# query -> 직장인 -> 거주자로 바꾸는 chain을 추가
dictionary = ["사람을 나타내는 표현 -> 거주자"]

prompt = ChatPromptTemplate.from_template(f"""
                사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요.
                만약 변경할 필요가 없다고 판단된다면, 사용자의 질문을 변경하지 않아도 됩니다.
                사전 : {dictionary}
                질문 : {{question}}
                """)

dictionary_chain = prompt | llm | StrOutputParser() 

tax_chain = {"query":dictionary_chain} | qa_chain

In [40]:
prompt = f"""[Identity]
- 당신은 최고의 한국 소득세 전문가입니다.
- [context]를 참고해서 사용자의 질문에 답변해주세요.

[context]
{retrieved_docs}

Question: {query}

"""

In [47]:
## RetrievalQA Chain을 어떻게 사용하는지 알아보자.
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever = database.as_retriever(),
    chain_type_kwargs = {"prompt":prompt}
    
)

In [43]:
ai_message = qa_chain({"query":query})

NameError: name 'qa_chain' is not defined

In [44]:
ai_message

AIMessage(content='연봉 5,000만원의 직장인이 납부해야 할 소득세를 계산해보겠습니다. 소득세는 종합소득 과세표준에 따라 세율이 적용됩니다.\n\n1. **연봉 5,000만원의 경우**:\n   - 5,000만원은 **1,400만원 초과 ~ 5,000만원 이하** 구간에 해당합니다.\n   - 이 구간의 세율은 다음과 같습니다:\n     - 1,400만원 이하: 과세표준 × 6%\n     - 1,400만원 초과 ~ 5,000만원 이하: 84만원 + (1,400만원 초과분 × 15%)\n\n2. **계산**:\n   - 5,000만원에서 1,400만원을 빼면 3,600만원이 남습니다.\n   - 따라서, 세액은 다음과 같이 계산됩니다:\n     - 기본 세액: 84만원\n     - 추가 세액: 3,600만원 × 15% = 540만원\n\n3. **총 소득세**:\n   - 총 소득세 = 기본 세액 + 추가 세액\n   - 총 소득세 = 84만원 + 540만원 = 624만원\n\n따라서, 연봉 5,000만원의 직장인은 약 **624만원**의 소득세를 납부해야 합니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 301, 'prompt_tokens': 5150, 'total_tokens': 5451, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'cha